In [2]:
import xarray as xr
import numpy as np
import sys, os, glob
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

os.chdir(root_dir)

from data_processing.libraries import mean_statistics, std_statistics, wind_power_density, weibull , weibull_statistics, wind_power_density, turbine_power, solar_power
from data_processing.libraries import regional_extraction

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from analysis.plotters import map_plotter, all_sample_points, variability_plotter, months, colors, calculate_color_levels, CERRA_target_grids
from data_processing.libraries import longitude_convert_0_to_360, find_nearest_indice, compute_statistics

In [5]:
client.close()
cluster.close()

In [2]:
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=8, threads_per_worker=2, memory_limit='2GB')
client = dd.Client(cluster)
print(client)

<Client: 'tcp://127.0.0.1:38281' processes=8 threads=16, memory=14.90 GiB>


In [6]:
run = 'New_runs'
case = 'Netherlands_coast'
file_name = '15MW/tp_150'
variable = 'power'

run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/statistics_files/{file_name}'

mean_ds = xr.open_dataset(f'{target_dir}/mean.nc')
std_ds = xr.open_dataset(f'{target_dir}/std.nc')

# read the target dataset if it exists
statistic = 'cov'
target_file = f'{target_dir}/{statistic}.nc'
if os.path.exists(target_file):
    with xr.open_dataset(target_file) as target_ds:
        target_ds = target_ds.load()
else:
    target_ds = xr.Dataset()
target_ds

<xarray.Dataset>
Dimensions:            (south_north: 199, west_east: 199, hour: 24, month: 12,
                        year: 31, season: 4)
Coordinates:
    num_z_levels_stag  float32 150.0
    XLONG              (south_north, west_east) float32 2.171 2.179 ... 3.642
    XLAT               (south_north, west_east) float32 50.94 50.94 ... 51.86
  * hour               (hour) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
  * month              (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year               (year) int64 1990 1991 1992 1993 ... 2017 2018 2019 2020
  * season             (season) object 'DJF' 'JJA' 'MAM' 'SON'
Dimensions without coordinates: south_north, west_east
Data variables:
    hourly_values      (hour, south_north, west_east) float32 0.7745 ... 0.7327
    monthly_values     (month, south_north, west_east) float32 0.7285 ... 0.6896
    yearly_values      (year, south_north, west_east) float32 0.8526 ... 0.732
    overall_values     (south_north, west_east) float32 0.8754 0.8741 ... 0.7763
    season_values      (season, south_north, west_east) float32 0.7493 ... 0.732

In [19]:
seasons = np.array(list(ds.groupby(f'Time.season').groups.keys()))
seasonal_values = np.zeros((2, len(ds.groupby(f'Time.season'))))
for season, seasonal_data in enumerate(ds.groupby(f'Time.season')):
    print(season)

0
1
2
3
